In [1]:
%pylab inline
import pandas as pd
import json
import time
pd.options.display.max_rows = 2000
import regex as re

from geopy.geocoders import Nominatim
geolocator = Nominatim()

from geopy.exc import GeocoderTimedOut

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)
    
from mtranslate import translate

Populating the interactive namespace from numpy and matplotlib


In [2]:
sprev=""
sprev_re=""
a=["á","ă","â"]
e=["é"]
i=["í","î"]
o=["ö","ó","ő"]
u=["ü","ú","ű"]
s=["ș"]
t=["ț"]
s_letters=[a,e,i,o,u,s,t]
s_Letters=["a","e","i","o","u","s","t"]

In [3]:
def do_match(wt,ws):
    """ wt is the one in which we will search ws is the searched one"""
    if len(wt)>0:
        length=len(ws)
        return ((re.search(r'(%s){e<=1}'%ws,wt) and length>6) or (re.search(r'(%s)'%ws,wt) and length<=6 and length>=4))
    else: return False

In [4]:
def replace_place(string):
    to_replace=['balan', 'barot', 'csikszereda', 'gyergyo', 'keresztur', 'kezdi',
       'kolozsvar', 'kovaszna', 'parajd', 'regen', 'segesvar', 'szentgyorgy',
       'szovata', 'toplita', 'udvarhely', 'vasarhely', 'vlahica']
    replacement=['Bălan', 'Baraolt', 'Miercurea Ciuc', 'Gheorgheni', 'Cristuru Secuiesc', 'Târgu Secuiesc',
       'Cluj-Napoca', 'Covasna', 'Praid', 'Reghin', 'Sighisoara', 'Sfântu Gheorghe',
       'Sovata', 'Toplița, Harghita', 'Odorheiu Secuiesc', 'Târgu Mureș', 'Vlahița']
    for i in arange(len(to_replace)):
        if string==to_replace[i]: return replacement[i]
    return string

In [5]:
def make_re(string):
    if(isinstance(string, str)):
        for i in arange(0,len(s_letters)):
            for c in s_letters[i]:
                if(c in string): 
                    string=string.replace(c,s_Letters[i])
        if('*' in string): string=string.replace('*','')
    return string

In [6]:
sprev=''
is_ro=False

s_cprev=''
cprev=''
is_c=False

s_szprev=''
szprev=''
is_sz=False

In [30]:
def find_country(string):
    global sprev
    global is_ro
    global cprev
    global s_cprev
    global is_c
    global szprev
    global s_szprev
    global is_sz
    if(isinstance(string, str)):
        string=make_re(string)
        
        for name in ["Romania"]:
            if do_match(string,name): 
                return "Romania"
        for name in ["Hungary","Magyarorszag"]:
            if do_match(string,name): 
                return "Hungary"
            
        if is_ro and do_match(string,sprev):
            return "Romania"   
        
        if is_c and do_match(string,cprev):
            return s_cprev 
        
        if is_sz and do_match(string,szprev):
            return s_szprev 
        
        for name in Countries:
            if isinstance(name,str) and do_match(string,name): 
                is_c=True
                cprev=string
                s_cprev='Country'
                return 'Country'
            
        for name in Szekely:
            if isinstance(name,str) and do_match(string,name): 
                is_sz=True
                szprev=string
                s_szprev= "Romania3"
                return "Romania3"
            
        for name in RO_211:
            if isinstance(name,str) and do_match(string,name): 
                is_ro=True
                sprev=string
                return "Romania2"
            
        return 'none'

In [31]:
data=pd.DataFrame.from_dict(json.loads(open("../first.json").read()))[:10000]#only first 5000 but these get the location where they live
data2=pd.DataFrame.from_dict(json.loads(open("../data.json").read()))#all the data, birth location add
ro_211=pd.read_excel('romania_telepulesek_211.xlsx')
RO_211=[]
for i in arange(0,len(ro_211["Ro"])):
    RO_211.append(make_re(ro_211.loc[i,"Ro"]))
    RO_211.append(make_re(ro_211.loc[i,"Hu"]))
    
countries=pd.read_excel('countries.xlsx')
Countries=[]
for i in arange(0,len(countries["Country"])):
    Countries.append(countries.loc[i,"Country"])
    Countries.append(make_re(countries.loc[i,"Country"]))

szekely=pd.read_excel('szekelyfold_telepulesek_560.xlsx')
Szekely=[]
for i in arange(0,len(szekely["Ro"])):
    Szekely.append(make_re(szekely.loc[i,"Hu"]))
    Szekely.append(make_re(szekely.loc[i,"Ro"]))

In [32]:
data.groupby(by="livesin").describe()

id                \
                                                     count          mean   
livesin                                                                    
Aachen, Germany                                        2.0  15286.000000   
Aalborg, Denmark                                       1.0  11121.000000   
Aalen                                                  1.0  13896.000000   
Aarhus, Denmark                                        2.0  14285.500000   
Abano Terme                                            1.0  12090.000000   
Abu Dhabi, United Arab Emirates                        3.0  15648.000000   
Acilia, Lazio, Italy                                   1.0  16053.000000   
Adjudeni, Iasi, Romania                                1.0   1803.000000   
Agerskov, Sonderjylland, Denmark                       1.0  12031.000000   
Agyagfalva, Harghita, Romania                          4.0  14759.500000   
Ahaus                                                  1.0  11448.000000   
Ajkarendek, Veszprem, Hungary                          1.0  13854.000000   
Ajnád, Harghita, Romania                               1.0   1374.000000   
Alba Iulia                                             6.0  14753.666667   
Albacete                                               2.0  14196.500000   
Albis, Covasna, Romania                                2.0  14608.500000   
Alcalá de Henares, Madrid                              1.0  18433.000000   
Aldoboly, Covasna, Romania                             1.0  14837.000000   
Alexandria, Virginia                                   1.0  12886.000000   
Alfalú, Harghita, Romania                              2.0  14822.000000   
Allershausen                                           1.0  18170.000000   
Alsóbóldogfalva, Harghita, Romania                     6.0  14621.666667   
Alsócsernáton, Covasna, Romania                        2.0  16715.000000   
Alsónémedi                                             2.0  13405.500000   
Alsósófalva, Harghita, Romania                         4.0  12683.000000   
Altbach                                                1.0  13980.000000   
Altdorf bei Nürnberg                                   1.0  14896.000000   
Althütte                                               1.0  17210.000000   
Alunis, Mures                                          1.0  11902.000000   
Alunis, Salaj, Romania                                 1.0  12023.000000   
Amantea                                                1.0  12076.000000   
Amriswil, Switzerland                                  1.0  13450.000000   
Amsterdam, Netherlands                                 6.0  13526.500000   
Ancona, Italy                                          1.0  12187.000000   
Anderlecht, Belgium                                    1.0  18595.000000   
Angyalos, Covasna, Romania                             3.0  11017.000000   
Ansbach, Germany                                       1.0  17266.000000   
Antwerp, Belgium                                       2.0   6817.500000   
Anzio, Italy                                           1.0  10451.000000   
Araci, Covasna, Romania                                1.0  17553.000000   
Arad, Arad                                            11.0  14031.818182   
Ardeal                                                 1.0  17466.000000   
Arenas de San Pedro, Spain                             1.0  12125.000000   
Arezzo, Italy                                          1.0  15658.000000   
Arkos                                                  8.0  13330.500000   
Arnsberg, Germany                                      1.0  18758.000000   
Ashford, Kent, England                                 1.0  12634.000000   
Assisi, Italy                                          1.0  10235.000000   
Astromeriti, Nicosia, Cyprus                           2.0  14248.000000   
Athens, Greece                                         2.0  16188.500000   
Atherstone, Warwickshire    

In [33]:
data.sort_values(by="livesin",inplace=True)
start=time.time()
data["livesin_country"]=data["livesin"].apply(find_country)
end=time.time()

In [34]:
print(end-start)

226.87576413154602


now 1000 profiles in 30

10000 profiles 240 and after adjustment it was 226

find_country: 1000 profiles in <42 s 
find_country: 
5000 profiles in 200 s-> more or less linear

In [28]:
data.groupby(by="livesin_country").count().sort_values(by="id")

,birth,id,livesin,marriedsince,marriedto,name,other1,other2,other3,other4,other5,other6,school,schoolyear,studwhat,studwhere,studyear,workwhat,workwhere
livesin_country,,,,,,,,,,,,,,,,,,,
Romania2,60,60,60,3,9,60,10,0,0,0,0,0,0,2,7,26,5,16,33
Hungary,461,461,461,53,81,461,71,3,0,0,0,0,1,16,34,185,48,94,234
Romania3,597,597,597,50,78,597,116,2,0,0,0,0,2,21,19,264,33,74,307
none,628,628,628,69,114,628,112,0,0,0,0,0,2,16,47,245,63,124,341
Country,728,728,728,81,133,728,116,7,0,0,0,0,2,18,46,299,61,132,379
Romania,6353,6353,6353,663,1088,6353,1050,37,0,0,0,0,16,170,529,2939,774,1285,3276


In [29]:
data[data["livesin_country"]=='none'].groupby(by="livesin").count()

,birth,id,marriedsince,marriedto,name,other1,other2,other3,other4,other5,other6,school,schoolyear,studwhat,studwhere,studyear,workwhat,workwhere,livesin_country
livesin,,,,,,,,,,,,,,,,,,,
Aalen,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1
Abano Terme,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1
Ahaus,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1
Albacete,2,2,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,2,2
"Alcalá de Henares, Madrid",1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Allershausen,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1
Alsónémedi,2,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2
Altbach,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1
Altdorf bei Nürnberg,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1


In [14]:
data["birth"]=data["birth"].apply(replace_place)
data2["birth"]=data2["birth"].apply(replace_place)

In [15]:
start=time.time()
for i in [0,2]:#birth livesin
    data.sort_values(by=data.columns[i],inplace=True,ascending=False)
    p_lat, p_lon, located_p=give_coordinates2(data[data.columns[i]])
    data[data.columns[i]+"_p_lat"]=p_lat
    data[data.columns[i]+"_p_lon"]=p_lon
    data[data.columns[i]+"_located"]=located_p
end=time.time()

NameError: name 'give_coordinates2' is not defined

In [ ]:
print(end-start)

In [ ]:
def get_countries(places):
    x=[]
    for place in places:
        if(place!=None):
            place_split=re.split("/|, ",place)
            x.append(place_split[len(place_split)-1])
        else: x.append(None)
    return x

Megtalalom az orszagot es leforditom angolra

In [ ]:
column="livesin_located"
data["NAME"]=get_countries(data[column])
data.sort_values(by="NAME",inplace=True,ascending=False)
data["NAME"]=[translate_s3(name) for name in data["NAME"]]

In [ ]:
all_data=pd.concat([all_data,data])

In [ ]:
all_data2 = all_data.drop_duplicates(subset='id')

In [ ]:
all_data2.describe()

In [ ]:
all_data2.to_json("second.json")